In [1]:
import numpy as np
import pandas as pd
from pandas import Series, DataFrame

results_df = pd.read_csv('Batch_121669_batch_results.csv')
results_df.head()

,HITId,HITTypeId,Title,Description,Keywords,Reward,CreationTime,MaxAssignments,RequesterAnnotation,AssignmentDurationInSeconds,...,RequesterFeedback,WorkTimeInSeconds,LifetimeApprovalRate,Last30DaysApprovalRate,Last7DaysApprovalRate,Input.image_name,Input.image_url,Answer.NumberOfItems,Approve,Reject
0,3IZVJEBJ6AKUSUW64YTOZUTAEDJ6Z2,3NB4XF0P1V96VV4KVNWTLITSILIL22,Transcribe text from an image,Submit the text shown in the given image,"image, write, transcription",$0.10,Wed Feb 10 14:05:28 PST 2016,3,BatchId:121669;,604800,...,NaN,16,0% (0/0),0% (0/0),0% (0/0),0.png,https://s3-us-west-2.amazonaws.com/ai2-vision/...,HAIR,NaN,NaN
1,3IZVJEBJ6AKUSUW64YTOZUTAEDJ6Z2,3NB4XF0P1V96VV4KVNWTLITSILIL22,Transcribe text from an image,Submit the text shown in the given image,"image, write, transcription",$0.10,Wed Feb 10 14:05:28 PST 2016,3,BatchId:121669;,604800,...,NaN,6,0% (0/0),0% (0/0),0% (0/0),0.png,https://s3-us-west-2.amazonaws.com/ai2-vision/...,hair,NaN,NaN
2,3IZVJEBJ6AKUSUW64YTOZUTAEDJ6Z2,3NB4XF0P1V96VV4KVNWTLITSILIL22,Transcribe text from an image,Submit the text shown in the given image,"image, write, transcription",$0.10,Wed Feb 10 14:05:28 PST 2016,3,BatchId:121669;,604800,...,NaN,25,0% (0/0),0% (0/0),0% (0/0),0.png,https://s3-us-west-2.amazonaws.com/ai2-vision/...,HIAR,NaN,NaN
3,3ZURAPD288NTGR49XGN016MBUX9F1P,3NB4XF0P1V96VV4KVNWTLITSILIL22,Transcribe text from an image,Submit the text shown in the given image,"image, write, transcription",$0.10,Wed Feb 10 14:05:28 PST 2016,3,BatchId:121669;,604800,...,NaN,4,0% (0/0),0% (0/0),0% (0/0),0.png,https://s3-us-west-2.amazonaws.com/ai2-vision/...,NOSE,NaN,NaN
4,3ZURAPD288NTGR49XGN016MBUX9F1P,3NB4XF0P1V96VV4KVNWTLITSILIL22,Transcribe text from an image,Submit the text shown in the given image,"image, write, transcription",$0.10,Wed Feb 10 14:05:28 PST 2016,3,BatchId:121669;,604800,...,NaN,9,0% (0/0),0% (0/0),0% (0/0),0.png,https://s3-us-west-2.amazonaws.com/ai2-vision/...,NosE,NaN,NaN


In [2]:
# Grab urls
image_urls = results_df['Input.image_url']

# Lowercase the answers
answers = results_df['Answer.NumberOfItems'].str.lower()

df = pd.concat([image_urls, answers], axis=1)
df.head()

,Input.image_url,Answer.NumberOfItems
0,https://s3-us-west-2.amazonaws.com/ai2-vision/...,hair
1,https://s3-us-west-2.amazonaws.com/ai2-vision/...,hair
2,https://s3-us-west-2.amazonaws.com/ai2-vision/...,hiar
3,https://s3-us-west-2.amazonaws.com/ai2-vision/...,nose
4,https://s3-us-west-2.amazonaws.com/ai2-vision/...,nose


In [3]:
# Grab image_url and convert to list
HIT_id = image_urls.values.tolist()

# Create dict with empty lists for each key
mydict = dict.fromkeys(HIT_id)

# Populate dict with responses as values
for key in mydict:
    
    # Create a list if there isn't one for the key
    if mydict[key] == None:
        mydict[key] = []
        
    # Add results to associated url key
    key_results = df['Answer.NumberOfItems'].loc[(df['Input.image_url'] == key)].values.tolist()
    for i in range(0, len(key_results)):
        mydict[key].append(key_results[i])
    
    # Use a dict to determine consensus
    countsdict = {x:mydict[key].count(x) for x in mydict[key]}
    
    # Iterate through keys in counting dict and replace with consensus value
    flag = False
    for response_key in countsdict:
        
        # Only count as consensus if >50%
        if countsdict[response_key] > (len(countsdict) / 2):
            flag = True
            mydict[key] = response_key

    if flag == False:
        mydict[key] = 'NO AGREEMENT'

In [4]:
mydict

{'https://s3-us-west-2.amazonaws.com/ai2-vision/example-turk/0.png-0.png': 'hair',
 'https://s3-us-west-2.amazonaws.com/ai2-vision/example-turk/0.png-1.png': 'nose',
 'https://s3-us-west-2.amazonaws.com/ai2-vision/example-turk/0.png-2.png': 'eyes',
 'https://s3-us-west-2.amazonaws.com/ai2-vision/example-turk/0.png-3.png': 'NO AGREEMENT',
 'https://s3-us-west-2.amazonaws.com/ai2-vision/example-turk/0.png-4.png': 'mouth',
 'https://s3-us-west-2.amazonaws.com/ai2-vision/example-turk/0.png-5.png': 'face',
 'https://s3-us-west-2.amazonaws.com/ai2-vision/example-turk/1.png-0.png': 'd',
 'https://s3-us-west-2.amazonaws.com/ai2-vision/example-turk/1.png-1.png': 'b',
 'https://s3-us-west-2.amazonaws.com/ai2-vision/example-turk/1.png-2.png': 'a',
 'https://s3-us-west-2.amazonaws.com/ai2-vision/example-turk/1.png-3.png': 'c',
 'https://s3-us-west-2.amazonaws.com/ai2-vision/example-turk/10.png-0.png': 'citrullus colocynthis',
 'https://s3-us-west-2.amazonaws.com/ai2-vision/example-turk/10.png-1.pn

In [5]:
import json

# Create an output filem 'w' for write
file_out = open('results.json','w')

# Write dictionary to file
json.dump(mydict,file_out, indent=4)                                    

# Close the file
file_out.close()